In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import scipy.ndimage
import os

import scipy.signal


def TTLtoTimes(TTL,Fs = 20000):
    
    if isinstance(TTL[0],(np.bool_,bool)):
        TTL = list(map(int,TTL))
    
    diff_TTL = np.diff(TTL)
    
    t_start = np.where(diff_TTL == 1)
    t_end = np.where(diff_TTL == -1)
    t_TTL = np.array([np.mean(interval,0) for interval in zip(t_start,t_end)])[0]
    
    return t_TTL/Fs

def speed(pos):
    
    x_speed = np.diff(pos['x'])/np.diff(pos.index)
    y_speed = np.diff(pos['y'])/np.diff(pos.index)
    
    v = np.sqrt(x_speed**2 + y_speed**2)
    
    return v

def load_pos():
    import pandas as pd
    files = os.listdir()
    for f in files: 
        if f.endswith('.h5'): 
            filename = f
    
    data = pd.read_hdf(filename)
    data = data[data.keys()[0][0]]
    return data 

def load_digitalin(channels,Fs = 20000):
    import pandas as pd
    
    digital_word = np.fromfile('digitalin.dat','uint16')
    sample = len(digital_word)
    time = np.arange(0,sample)
    time = time/Fs

    chan = {}
    
    for i in range(len(channels)):
        chan.update({channels[i]:(digital_word & 2**i)>0})

    return time,chan

path = 'Y:/ExampleSession/rat01_200630_114732'
path = 'Y:/ExampleSession/rat01_200630_171900'
path = 'Y:/ExampleSession/rat01_200630_183149/'

path = 'Y:/ExampleSession/RAT02/rat02_200707_114611'


i = 1
title = ['Fear','Tone','Context']

os.chdir(path)
step = 100

time,data = load_digitalin(['TTL','Shock','Tone'])
pos = load_pos()

s = speed(pos['snout'])
s = scipy.ndimage.gaussian_filter1d(s,1)

tf = TTLtoTimes(data['TTL'])
tf = np.delete(tf,-1)

pos.index = tf
time = time[0:-1:step]

fig = plt.subplot(1,1,i)
plt.tight_layout()
plt.title(title[i-1])
plt.plot(time,data['TTL'][0:-1:step])
plt.plot(time,data['Shock'][0:-1:step] * 3)
plt.plot(time,data['Tone'][0:-1:step]*2)
plt.plot(pos.index[:-1],s/15)
plt.legend(['TTL','Shock','Tone','Speed'])
plt.xlabel('Time (s)')
#plt.vlines(x=tf,ymin = 0, ymax = 3,color='red')
i += 1

plt.show()